In [101]:
import tifffile
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from natsort import natsorted
from skimage.transform import resize
from tqdm.auto import tqdm
from skimage.measure import label
import tifffile

In [182]:
mask_dirs = natsorted(glob("masks/*"))[:1000]
image_dirs = natsorted(glob("convolutions/*"))[:1000]

In [191]:
n_in_tile = 25
tile_width = int(np.sqrt(n_in_tile))
grid_shape = (tile_width, tile_width)

In [192]:
mask_tuples = [tuple(mask_dirs[i:i+n_in_tile]) for i in range(0, len(mask_dirs), n_in_tile)]
img_tuples = [tuple(image_dirs[i:i+n_in_tile]) for i in range(0, len(image_dirs), n_in_tile)]

In [193]:
mask_tuple = mask_tuples[0]
masks = [np.array(Image.open(dir_)) for dir_ in mask_tuple]

In [194]:
arrays = [np.array(Image.open(dir_)) for dir_ in mask_tuple]
array_grid = np.array(arrays).reshape(grid_shape + arrays[0].shape)
result = np.block([[array_grid[i, j] for j in range(grid_shape[1])] for i in range(grid_shape[0])])

In [195]:
for i, (mask_tuple, img_tuple) in enumerate(zip(tqdm(mask_tuples), img_tuples)):

    arrays = [np.array(Image.open(dir_)) for dir_ in mask_tuple]
    array_grid = np.array(arrays).reshape(grid_shape + arrays[0].shape)
    result = np.block([[array_grid[i, j] for j in range(grid_shape[1])] for i in range(grid_shape[0])])
    result = label(result).astype(np.uint16)
    Image.fromarray(result).save(f"omnipose_td/{str(i).zfill(4)}_masks.png")
    Image.fromarray(result).save(f"omnipose_td_pretrained/{str(i).zfill(4)}_masks.png")

    arrays = [np.array(Image.open(dir_)) for dir_ in img_tuple]
    array_grid = np.array(arrays).reshape(grid_shape + arrays[0].shape)
    result = np.block([[array_grid[i, j] for j in range(grid_shape[1])] for i in range(grid_shape[0])])
    Image.fromarray(result).save(f"omnipose_td/{str(i).zfill(4)}.png")
    Image.fromarray(result).save(f"omnipose_td_pretrained/{str(i).zfill(4)}.png")

  0%|          | 0/40 [00:00<?, ?it/s]